In [ ]:
# Unmount whatever is there:
from google.colab import drive
drive.flush_and_unmount()

# Now /content/drive should be empty. You can safely mount again:
drive.mount('/content/drive')



Drive not mounted, so nothing to flush and unmount.


ValueError: Mountpoint must not already contain files

In [ ]:
# ----------------------------------------------------------
# MAMI2022 Preprocessing + Dataset Loader (with file checks)
# ----------------------------------------------------------
import os
import pandas as pd
import numpy as np
from PIL import Image, ImageEnhance
import cv2

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# ----------------------------------------
# 1. Image Preprocessing Function
# ----------------------------------------
def preprocess_image(
    img_path: str,
    target_size=(384, 384),
    enhance_contrast=1.5,
    enhance_sharpness=2.0,
    apply_noise_removal=False
) -> torch.Tensor:
    """
    Load and preprocess a single meme image for UniFairNet.
    Steps:
      1. Load image in RGB.
      2. Resize with aspect-ratio padding to target_size.
      3. Apply contrast and sharpness enhancement.
      4. Optionally apply noise removal (median blur).
      5. Return a normalized tensor ready for CLIP/ViT input.
    """
    # 1. Load
    image = Image.open(img_path).convert('RGB')
    original_w, original_h = image.size
    target_w, target_h = target_size

    # 2. Resize with aspect-ratio padding
    ratio = min(target_w / original_w, target_h / original_h)
    new_w = int(original_w * ratio)
    new_h = int(original_h * ratio)
    image_resized = image.resize((new_w, new_h), Image.BICUBIC)

    # Paste centered onto a black canvas of size target_size
    canvas = Image.new('RGB', target_size, (0, 0, 0))
    paste_x = (target_w - new_w) // 2
    paste_y = (target_h - new_h) // 2
    canvas.paste(image_resized, (paste_x, paste_y))

    # 3. Contrast & Sharpness Enhancement
    contrast_enhancer = ImageEnhance.Contrast(canvas)
    img_contrasted = contrast_enhancer.enhance(enhance_contrast)
    sharpness_enhancer = ImageEnhance.Sharpness(img_contrasted)
    img_sharp = sharpness_enhancer.enhance(enhance_sharpness)

    # 4. Optional Noise Removal
    if apply_noise_removal:
        img_np = cv2.cvtColor(np.array(img_sharp), cv2.COLOR_RGB2BGR)
        img_np = cv2.medianBlur(img_np, 3)
        img_processed = Image.fromarray(cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB))
    else:
        img_processed = img_sharp

    # 5. Normalize to tensor
    normalize_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                             std=[0.5, 0.5, 0.5])
    ])
    tensor_out = normalize_transform(img_processed)
    return tensor_out


# ----------------------------------------
# 2. MAMI2022 PyTorch Dataset Class (with file existence check)
# ----------------------------------------
class MAMIDataset(Dataset):
    """
    PyTorch Dataset for MAMI2022, which drops any TSV rows
    whose referenced image file does not actually exist.

    Expects directory structure:

      <root_dir>/
        ├── train.tsv
        ├── validation.tsv
        ├── test.tsv
        └── MAMI_2022_images/
            ├── training_images/
            └── test_images/

    Each .tsv has columns:
      file_name    label    shaming    stereotype    objectification    violence    text
    We take the four sub-labels [shaming, stereotype, objectification, violence]
    as our multi-label target.
    """

    def __init__(
        self,
        root_dir: str,
        split: str = "train",
        target_size=(384, 384),
        enhance_contrast=1.5,
        enhance_sharpness=2.0,
        apply_noise_removal=False
    ):
        super(MAMIDataset, self).__init__()
        assert split in ["train", "validation", "test"], \
            "split must be one of ['train','validation','test']"

        self.root_dir = root_dir
        self.split = split
        self.target_size = target_size
        self.enhance_contrast = enhance_contrast
        self.enhance_sharpness = enhance_sharpness
        self.apply_noise_removal = apply_noise_removal

        # 2.1 Load TSV into DataFrame
        ann_path = os.path.join(root_dir, f"{split}.tsv")
        df = pd.read_csv(ann_path, sep="\t")

        # 2.2 Figure out image folder path
        images_folder = os.path.join(root_dir, "MAMI_2022_images")
        if split in ["train", "validation"]:
            img_dir = os.path.join(images_folder, "training_images")
        else:  # "test"
            img_dir = os.path.join(images_folder, "test_images")

        # 2.3 Filter out any rows whose image file is missing
        valid_rows = []
        for _, row in df.iterrows():
            img_file = row["file_name"]
            full_path = os.path.join(img_dir, img_file)
            if os.path.isfile(full_path):
                valid_rows.append(row)
            else:
                # If you’d like, print a warning for missing files:
                # print(f"Warning: {full_path} not found, skipping.")
                pass

        # Re‐build DataFrame using only valid rows
        self.df = pd.DataFrame(valid_rows).reset_index(drop=True)
        self.img_dir = img_dir

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        filename = row["file_name"]
        img_path = os.path.join(self.img_dir, filename)

        # 1. Preprocess the image
        image_tensor = preprocess_image(
            img_path,
            target_size=self.target_size,
            enhance_contrast=self.enhance_contrast,
            enhance_sharpness=self.enhance_sharpness,
            apply_noise_removal=self.apply_noise_removal
        )

        # 2. Prepare multi-label vector [shaming, stereotype, objectification, violence]
        labels = torch.tensor([
            row["shaming"],
            row["stereotype"],
            row["objectification"],
            row["violence"]
        ], dtype=torch.float)

        # 3. Return (image_tensor, labels, raw_text)
        return image_tensor, labels, row["text"]


# ----------------------------------------
# 3. Example DataLoader Instantiation
# ----------------------------------------
if __name__ == "__main__":
    # 3.1 Adjust this to your actual MAMI2022 folder location:
    DATA_ROOT = "/content/drive/MyDrive/MAMI2022"  # <— replace with your own path

    # 3.2 Instantiate datasets
    train_dataset = MAMIDataset(
        root_dir=DATA_ROOT,
        split="train",
        target_size=(384, 384),
        enhance_contrast=1.5,
        enhance_sharpness=2.0,
        apply_noise_removal=True
    )
    val_dataset = MAMIDataset(root_dir=DATA_ROOT, split="validation")
    test_dataset = MAMIDataset(root_dir=DATA_ROOT, split="test")

    # 3.3 DataLoaders (Limit workers to 2 to avoid DataLoader warnings)
    train_loader = DataLoader(
        train_dataset,
        batch_size=16,
        shuffle=True,
        num_workers=2   # ≤ recommended max on Colab
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=16,
        shuffle=False,
        num_workers=2
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=16,
        shuffle=False,
        num_workers=2
    )

    # 3.4 Sanity-check one batch
    for batch_images, batch_labels, batch_texts in train_loader:
        print("Batch image shape:", batch_images.shape)   # e.g. [16, 3, 384, 384]
        print("Batch labels shape:", batch_labels.shape)   # e.g. [16, 4]
        print("Example raw text:", batch_texts[0])         # string from TSV
        break

    print("MAMI2022 preprocessing + loader test complete.")


Batch image shape: torch.Size([16, 3, 384, 384])
Batch labels shape: torch.Size([16, 4])
Example raw text: GRAVITY Not just a rule, it's the law.
MAMI2022 preprocessing + loader test complete.


In [ ]:
# ------------------------------------------------------------------------
# MAMI2022 Full Dataset Preprocessing & Saving Script
# ------------------------------------------------------------------------
# This script:
#  1. Loads MAMI2022 annotations (train/validation/test .tsv files).
#  2. Preprocesses each image (resize, pad, enhance, denoise, normalize).
#  3. Saves each preprocessed image tensor to disk as a .pt file.
#  4. Saves a single metadata file per split containing:
#        • A list of all image‐.pt filenames
#        • Corresponding multi‐label targets [shaming, stereotype, objectification, violence]
#        • Original text captions (optional, for debugging or OCR comparison)
#
# After running, you will have a “processed” folder:
#   <OUTPUT_ROOT>/
#     ├── train/
#     │     ├── images/          ← preprocessed .pt tensors, named by original filename
#     │     └── metadata.pt      ← dict with keys 'filenames', 'labels', 'texts'
#     ├── validation/
#     │     ├── images/
#     │     └── metadata.pt
#     └── test/
#           ├── images/
#           └── metadata.pt
#
# In future training, you can load each split by:
#   metadata = torch.load("<OUTPUT_ROOT>/train/metadata.pt")
#   for fn, label in zip(metadata['filenames'], metadata['labels']):
#       tensor = torch.load("<OUTPUT_ROOT>/train/images/" + fn + ".pt")
#       ...
#
# Usage:
#   1. Adjust DATA_ROOT to point to where your MAMI2022 folder lives.
#   2. Adjust OUTPUT_ROOT to where you want to store processed tensors.
#   3. Run this entire script; it will preprocess & save data.
# ------------------------------------------------------------------------

import os
import pandas as pd
import numpy as np
from PIL import Image, ImageEnhance
import cv2

import torch
from torch.utils.data import Dataset
from torchvision import transforms
from tqdm import tqdm

# ----------------------------------------
# 1. Image Preprocessing Function
# ----------------------------------------
def preprocess_image(
    img_path: str,
    target_size=(384, 384),
    enhance_contrast=1.5,
    enhance_sharpness=2.0,
    apply_noise_removal=True
) -> torch.Tensor:
    """
    Load and preprocess a single meme image for UniFairNet.

    Steps:
      1. Load image in RGB.
      2. Resize with aspect-ratio padding to target_size.
      3. Apply contrast and sharpness enhancement.
      4. Optionally apply noise removal (median blur).
      5. Return a normalized tensor (3 x H x W, range [-1, +1]).
    """
    # 1. Load
    image = Image.open(img_path).convert('RGB')
    original_w, original_h = image.size
    target_w, target_h = target_size

    # 2. Resize with aspect-ratio padding
    ratio = min(target_w / original_w, target_h / original_h)
    new_w = int(original_w * ratio)
    new_h = int(original_h * ratio)
    image_resized = image.resize((new_w, new_h), Image.BICUBIC)

    # Create a black canvas, paste centered
    canvas = Image.new('RGB', target_size, (0, 0, 0))
    paste_x = (target_w - new_w) // 2
    paste_y = (target_h - new_h) // 2
    canvas.paste(image_resized, (paste_x, paste_y))

    # 3. Contrast & Sharpness Enhancement
    contrast_enhancer = ImageEnhance.Contrast(canvas)
    img_contrasted = contrast_enhancer.enhance(enhance_contrast)

    sharpness_enhancer = ImageEnhance.Sharpness(img_contrasted)
    img_sharp = sharpness_enhancer.enhance(enhance_sharpness)

    # 4. Optional Noise Removal
    if apply_noise_removal:
        img_np = cv2.cvtColor(np.array(img_sharp), cv2.COLOR_RGB2BGR)
        img_np = cv2.medianBlur(img_np, 3)
        img_processed = Image.fromarray(cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB))
    else:
        img_processed = img_sharp

    # 5. Normalize to tensor for CLIP/ViT (range [-1, +1])
    normalize_transform = transforms.Compose([
        transforms.ToTensor(),                                  # [0,1]
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                             std=[0.5, 0.5, 0.5])              # → [-1,+1]
    ])
    tensor_out = normalize_transform(img_processed)            # shape: [3, H, W]
    return tensor_out


# ----------------------------------------
# 2. Process & Save Entire Split
# ----------------------------------------
def process_split(
    split: str,
    data_root: str,
    output_root: str,
    target_size=(384, 384),
    enhance_contrast=1.5,
    enhance_sharpness=2.0,
    apply_noise_removal=True
):
    """
    Processes all images in one split (train, validation, or test),
    saving each preprocessed tensor to disk and collecting metadata.

    Args:
      split (str): one of 'train', 'validation', 'test'
      data_root (str): path to MAMI2022 root folder
      output_root (str): where to write processed data
    """
    assert split in ["train", "validation", "test"], "split must be one of train, validation, test"

    # 2.1. Paths
    tsv_path = os.path.join(data_root, f"{split}.tsv")
    df = pd.read_csv(tsv_path, sep="\t")

    images_folder = os.path.join(data_root, "MAMI_2022_images")
    if split in ["train", "validation"]:
        img_dir = os.path.join(images_folder, "training_images")
    else:
        img_dir = os.path.join(images_folder, "test_images")

    # 2.2. Prepare output directories
    split_out_dir = os.path.join(output_root, split)
    images_out_dir = os.path.join(split_out_dir, "images")
    os.makedirs(images_out_dir, exist_ok=True)

    # 2.3. Metadata lists
    filenames = []   # original file_name, without extension
    labels = []      # [shaming, stereotype, objectification, violence]
    texts = []       # raw text captions

    # 2.4. Iterate through rows, preprocess & save
    for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing {split}"):
        fname = row["file_name"]
        full_img_path = os.path.join(img_dir, fname)

        # Skip if image missing
        if not os.path.isfile(full_img_path):
            # Optionally: print(f"[WARNING] Missing {full_img_path}, skipping.")
            continue

        # Preprocess image → tensor
        tensor = preprocess_image(
            full_img_path,
            target_size=target_size,
            enhance_contrast=enhance_contrast,
            enhance_sharpness=enhance_sharpness,
            apply_noise_removal=apply_noise_removal
        )

        # Prepare metadata
        base_name = os.path.splitext(fname)[0]
        filenames.append(base_name)
        labels.append([int(row["shaming"]),
                       int(row["stereotype"]),
                       int(row["objectification"]),
                       int(row["violence"])])
        texts.append(row["text"])

        # Save tensor to disk as .pt
        out_path = os.path.join(images_out_dir, base_name + ".pt")
        torch.save(tensor, out_path)

    # 2.5. Save metadata as a single .pt file
    metadata = {
        "filenames": filenames,           # list of strings
        "labels": torch.tensor(labels),   # shape: [N, 4]
        "texts": texts                    # list of strings
    }
    torch.save(metadata, os.path.join(split_out_dir, "metadata.pt"))


# ----------------------------------------
# 3. Main: Process All Splits & Save
# ----------------------------------------
if __name__ == "__main__":
    # 3.1. Set your MAMI2022 root directory (where train.tsv etc. live)
    DATA_ROOT = "/content/drive/MyDrive/MAMI2022"  # <-- adjust to your path

    # 3.2. Set where you want processed data saved
    OUTPUT_ROOT = "/content/drive/MyDrive/MAMI2022_processed"  # <-- adjust

    # 3.3. Ensure output root exists
    os.makedirs(OUTPUT_ROOT, exist_ok=True)

    # 3.4. Process each split
    for split_name in ["train", "validation", "test"]:
        process_split(
            split=split_name,
            data_root=DATA_ROOT,
            output_root=OUTPUT_ROOT,
            target_size=(384, 384),
            enhance_contrast=1.5,
            enhance_sharpness=2.0,
            apply_noise_removal=True
        )

    print("All splits processed and saved to:", OUTPUT_ROOT)


Processing test: 100%|██████████| 1000/1000 [10:00<00:00,  1.66it/s]

All splits processed and saved to: /content/drive/MyDrive/MAMI2022_processed


In [ ]:
# =============================================================================
# Hateful Memes: Full-Dataset Preprocessing Script
#
# This script will:
#   1. Read train.jsonl, dev.jsonl, test.jsonl from the Hateful Memes folder.
#   2. Preprocess each image (resize, pad, enhance, denoise, normalize).
#   3. Save each preprocessed image tensor as “<basename>.pt” under a split-specific subfolder.
#   4. Save a metadata.pt for each split containing:
#         • filenames (list of image basenames)
#         • labels (torch.Tensor of 0/1)
#         • texts  (list of captions)
#
# After running, you’ll have a directory structure like:
#
#   <OUTPUT_ROOT>/
#     ├── train/
#     │     ├── images/          ← contains “000001.pt”, “000002.pt”, …
#     │     └── metadata.pt      ← dict with keys "filenames", "labels", "texts"
#     ├── dev/
#     │     ├── images/
#     │     └── metadata.pt
#     └── test/
#           ├── images/
#           └── metadata.pt
#
# To run:
#   1. Mount or ensure Google Drive is accessible if using Colab.
#   2. Adjust DATA_ROOT to the folder containing img/, train.jsonl, dev.jsonl, test.jsonl.
#   3. Adjust OUTPUT_ROOT to the folder where you want to save processed data.
#   4. Run this script cell. It will print progress and save all tensors + metadata.
# =============================================================================

import os
import json
from PIL import Image, ImageEnhance
import cv2
import torch
import numpy as np
from torchvision import transforms
from tqdm import tqdm


# -----------------------------
# 1) CONFIGURE THESE PATHS
# -----------------------------
# FOLDER containing:
#    - img/           (all .jpg images, e.g. “000001.jpg”)
#    - train.jsonl
#    - dev.jsonl
#    - test.jsonl
DATA_ROOT = "/content/drive/MyDrive/HateNidhi/HatefulMemes/"

# FOLDER where preprocessed images + metadata will be saved
OUTPUT_ROOT = "/content/drive/MyDrive/HateNidhi/HatefulMemes/HatefulMemes_processed/"

# -----------------------------
# 2) PREPROCESSING FUNCTION
# -----------------------------
def preprocess_image(
    img_path: str,
    target_size=(384, 384),
    enhance_contrast=1.5,
    enhance_sharpness=2.0,
    apply_noise_removal=True
) -> torch.Tensor:
    """
    Load a single meme image, resize + pad to `target_size`, enhance contrast + sharpness,
    optionally apply median blur, then normalize to a tensor in [-1, +1].
    Returns a torch.Tensor of shape [3, H, W].
    """
    # 1. Load image
    image = Image.open(img_path).convert("RGB")
    orig_w, orig_h = image.size
    tgt_w, tgt_h = target_size

    # 2. Compute resize scale and pad offsets
    scale = min(tgt_w / orig_w, tgt_h / orig_h)
    new_w = int(orig_w * scale)
    new_h = int(orig_h * scale)
    image_resized = image.resize((new_w, new_h), Image.BICUBIC)

    canvas = Image.new("RGB", target_size, (0, 0, 0))
    paste_x = (tgt_w - new_w) // 2
    paste_y = (tgt_h - new_h) // 2
    canvas.paste(image_resized, (paste_x, paste_y))

    # 3. Enhance contrast + sharpness
    contrast_enhancer = ImageEnhance.Contrast(canvas)
    im_contrasted = contrast_enhancer.enhance(enhance_contrast)
    sharpness_enhancer = ImageEnhance.Sharpness(im_contrasted)
    im_sharp = sharpness_enhancer.enhance(enhance_sharpness)

    # 4. Optional median blur
    if apply_noise_removal:
        np_im = cv2.cvtColor(np.array(im_sharp), cv2.COLOR_RGB2BGR)
        np_im = cv2.medianBlur(np_im, 3)
        im_processed = Image.fromarray(cv2.cvtColor(np_im, cv2.COLOR_BGR2RGB))
    else:
        im_processed = im_sharp

    # 5. Normalize to tensor (range [-1.0, +1.0])
    normalize = transforms.Compose([
        transforms.ToTensor(),                             # [0,1]
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
                             std=[0.5, 0.5, 0.5])          # → [-1,+1]
    ])
    tensor = normalize(im_processed)                      # shape: [3, H, W]
    return tensor

# -----------------------------
# 3) SPLIT PROCESSING FUNCTION
# -----------------------------
def process_split(
    split: str,
    data_root: str,
    output_root: str,
    target_size=(384, 384),
    enhance_contrast=1.5,
    enhance_sharpness=2.0,
    apply_noise_removal=True
):
    """
    Processes all images in one split: 'train', 'dev', or 'test'.
    For each JSONL line, reads 'img' (filename), 'label', and 'text'.
    Skips any image if missing. Saves preprocessed tensor + collects metadata.
    """
    assert split in ["train", "dev", "test"], "split must be 'train', 'dev', or 'test'"

    # 3.1. Load JSONL
    jsonl_path = os.path.join(data_root, f"{split}.jsonl")
    print(jsonl_path)
    if not os.path.isfile(jsonl_path):
        raise FileNotFoundError(f"Cannot find {split}.jsonl in {data_root}")

    # 3.2. Paths
    img_dir = data_root
    print(img_dir)
    split_out = os.path.join(output_root, split)
    print(split)
    images_out = os.path.join(split_out, "images")
    os.makedirs(images_out, exist_ok=True)
    print(images_out)

    filenames = []
    labels = []
    texts = []

    # 3.3. Iterate and preprocess
    with open(jsonl_path, "r", encoding="utf-8") as f:
        counter = 0
        max_to_print = 5
        for line in tqdm(f, desc=f"Preprocessing {split}", unit="img"):
            record = json.loads(line)
            img_filename = record.get("img") or record.get("img_fn")


            if counter < max_to_print:
              print(img_filename)
              counter += 1

            label = int(record.get("label", 0))
            text = record.get("text", "") or record.get("text_only", "")

            src_path = os.path.join(img_dir, img_filename)
            if counter < max_to_print:
              print("SRC", src_path)
            if not os.path.isfile(src_path):
                # print("SKIPAROOOOOOOOO")
                # Skip missing files silently
                continue

            # Preprocess and save tensor
            tensor = preprocess_image(
                src_path,
                target_size=target_size,
                enhance_contrast=enhance_contrast,
                enhance_sharpness=enhance_sharpness,
                apply_noise_removal=apply_noise_removal
            )

            base_name = os.path.splitext(img_filename)[0]
            out_path = os.path.join(images_out, base_name + ".pt")

            print("Count",counter)
            if counter < max_to_print:
              print("OUT",out_path)
              print("BASE",base_name)

            if not os.path.exists(out_path):
              os.makedirs(os.path.dirname(out_path), exist_ok=True)

            torch.save(tensor, out_path)

            filenames.append(base_name)
            labels.append(label)
            texts.append(text)

    # 3.4. Save metadata
    meta = {
        "filenames": filenames,            # e.g. ["000001", "000002", …]
        "labels": torch.tensor(labels),    # shape [N], dtype=torch.long
        "texts": texts                     # list of captions
    }
    torch.save(meta, os.path.join(split_out, "metadata.pt"))


# -----------------------------
# 4) MAIN: PROCESS ALL SPLITS
# -----------------------------
if __name__ == "__main__":
    # 4.1. Quick check: does img/ exist and contain files?
    img_check = os.path.join(DATA_ROOT, "img")
    if not os.path.isdir(img_check) or len(os.listdir(img_check)) == 0:
        raise FileNotFoundError(
            f"Folder '{img_check}' either does not exist or is empty. "
            "Place your .jpg files in that directory and re-run."
        )

    # 4.2. Ensure OUTPUT_ROOT exists
    os.makedirs(OUTPUT_ROOT, exist_ok=True)

    # 4.3. Preprocess each split
    for split_name in ["train", "dev", "test"]:
        process_split(
            split=split_name,
            data_root=DATA_ROOT,
            output_root=OUTPUT_ROOT,
            target_size=(384, 384),
            enhance_contrast=1.5,
            enhance_sharpness=2.0,
            apply_noise_removal=True
        )

    print("✅ All splits processed. Saved under:", OUTPUT_ROOT)


/content/drive/MyDrive/HateNidhi/HatefulMemes/train.jsonl
/content/drive/MyDrive/HateNidhi/HatefulMemes/
train
/content/drive/MyDrive/HateNidhi/HatefulMemes/HatefulMemes_processed/train/images


Preprocessing train: 0img [00:00, ?img/s]

img/42953.png
SRC /content/drive/MyDrive/HateNidhi/HatefulMemes/img/42953.png


Preprocessing train: 1img [00:00,  1.24img/s]

Count 1
OUT /content/drive/MyDrive/HateNidhi/HatefulMemes/HatefulMemes_processed/train/images/img/42953.pt
BASE img/42953
img/23058.png
SRC /content/drive/MyDrive/HateNidhi/HatefulMemes/img/23058.png


...

Count 5
Count 5
Count 5


Preprocessing test: 999img [01:20, 11.62img/s]

Count 5
Count 5
Count 5


Preprocessing test: 1000img [01:21, 12.33img/s]

Count 5
✅ All splits processed. Saved under: /content/drive/MyDrive/HateNidhi/HatefulMemes/HatefulMemes_processed/


In [ ]:
import os
import torch
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
from torchvision import transforms
from tqdm import tqdm
import urllib.parse

# -------------------------------------------------------------
#  1) Paths – adjust these if your Drive layout is different
# -------------------------------------------------------------
DRIVE_ROOT      = "/content/drive/MyDrive"
DATASET_ROOT    = os.path.join(DRIVE_ROOT, "Memotion2")
IMAGES_FOLDER   = os.path.join(DATASET_ROOT, "image")          # Unzipped folder of all .jpg/.webp/.png files
CSV_FILE        = os.path.join(DATASET_ROOT, "memotion_train.csv")
PROCESSED_ROOT  = os.path.join(DRIVE_ROOT, "Memotion2_processed")

# Ensure output folders exist
os.makedirs(PROCESSED_ROOT, exist_ok=True)
os.makedirs(os.path.join(PROCESSED_ROOT, "images"), exist_ok=True)

# -------------------------------------------------------------
#  2) Read CSV metadata
# -------------------------------------------------------------
print("Loading CSV:", CSV_FILE)
df = pd.read_csv(CSV_FILE)

required_cols = [
    "Image url",
    "ocr_text",
    "humour",
    "sarcastic",
    "offensive",
    "motivational",
    "overall_sentiment"
]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns in CSV: {missing}")

url_list         = df["Image url"].astype(str).tolist()
ocr_texts        = df["ocr_text"].astype(str).tolist()
humour_list      = df["humour"].astype(str).tolist()
sarcastic_list   = df["sarcastic"].astype(str).tolist()
offensive_list   = df["offensive"].astype(str).tolist()
motivational_list= df["motivational"].astype(str).tolist()
sentiment_list   = df["overall_sentiment"].astype(str).tolist()

N = len(df)
print(f"Found {N} rows in CSV.")

# -------------------------------------------------------------
#  3) Build mapping dictionaries for each categorical column
# -------------------------------------------------------------
def build_mapping(str_list):
    uniques = sorted(set(str_list))
    return { s:i for i, s in enumerate(uniques) }

humour_map       = build_mapping(humour_list)
sarcastic_map    = build_mapping(sarcastic_list)
offensive_map    = build_mapping(offensive_list)
motivational_map = build_mapping(motivational_list)
sentiment_map    = build_mapping(sentiment_list)

print("humour_map:", humour_map)
print("sarcastic_map:", sarcastic_map)
print("offensive_map:", offensive_map)
print("motivational_map:", motivational_map)
print("sentiment_map:", sentiment_map)

# -------------------------------------------------------------
#  4) Image Transform (resize → 384×384, ToTensor, normalize)
# -------------------------------------------------------------
transform = transforms.Compose([
    transforms.Resize((384, 384), interpolation=Image.BICUBIC),
    transforms.ToTensor(),  # [0,1]
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet mean
        std =[0.229, 0.224, 0.225]   # ImageNet std
    )
])

# -------------------------------------------------------------
#  5) Loop over each row, load (local or URL), preprocess, save .pt
# -------------------------------------------------------------
error_count = 0
filenames   = []
label_matrix = torch.zeros((N, 5), dtype=torch.long)

for idx in tqdm(range(N), desc="Preprocessing images", unit="img"):
    # 5.1 Build a zero-padded base name, e.g. "0000001"
    base_name = str(idx+1).zfill(len(str(N)))
    filenames.append(base_name)

    # 5.2 Map each categorical label to its integer ID
    h_id    = humour_map[humour_list[idx]]
    s_id    = sarcastic_map[sarcastic_list[idx]]
    o_id    = offensive_map[offensive_list[idx]]
    m_id    = motivational_map[motivational_list[idx]]
    sen_id  = sentiment_map[sentiment_list[idx]]
    label_matrix[idx, :] = torch.tensor([h_id, s_id, o_id, m_id, sen_id], dtype=torch.long)

    # 5.3 Attempt to load the image from the local folder first
    #     Extract filename from the URL (strip query-strings)
    parsed_url = urllib.parse.urlparse(url_list[idx])
    local_filename = os.path.basename(parsed_url.path)  # e.g. "07ZcrjZ_d.webp"
    local_path = os.path.join(IMAGES_FOLDER, local_filename)

    img_pil = None
    if os.path.isfile(local_path):
        try:
            img_pil = Image.open(local_path).convert("RGB")
        except Exception as e:
            # If for some reason PIL cannot open it, fall back to download
            img_pil = None

    # 5.4 If local load failed, attempt to download from URL (with timeout)
    if img_pil is None:
        try:
            response = requests.get(url_list[idx], timeout=5)
            response.raise_for_status()
            img_pil = Image.open(BytesIO(response.content)).convert("RGB")
        except Exception as e:
            # Skip on any download/open error
            error_count += 1
            continue

    # 5.5 Apply transforms (resize → 384×384, normalize, etc.)
    img_t = transform(img_pil)  # Tensor [3,384,384]

    # 5.6 Save to .pt under PROCESSED_ROOT/images/<base_name>.pt
    out_path = os.path.join(PROCESSED_ROOT, "images", f"{base_name}.pt")
    torch.save(img_t, out_path)

print("Image preprocessing complete.")
if error_count > 0:
    print(f"Warning: {error_count} images failed to load (skipped).")

# -------------------------------------------------------------
#  6) Save metadata.pt
# -------------------------------------------------------------
metadata = {
    "filenames": filenames,             # ["0000001", "0000002", …]
    "labels":    label_matrix,          # LongTensor shape [N,5]
    "ocr_texts": ocr_texts,             # list of length N
    "urls":      url_list,              # list of length N
    "mappings": {
        "humour":       humour_map,
        "sarcastic":    sarcastic_map,
        "offensive":    offensive_map,
        "motivational": motivational_map,
        "sentiment":    sentiment_map
    }
}
torch.save(metadata, os.path.join(PROCESSED_ROOT, "metadata.pt"))
print("Saved metadata.pt with keys:", list(metadata.keys()))


Loading CSV: /content/drive/MyDrive/Memotion2/memotion_train.csv


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Memotion2/memotion_train.csv'

In [ ]:
!ls /content/drive/MyDrive/HatefulMemes_processed/

dev  test  train


In [ ]:
!ls /content/drive/MyDrive/HatefulMemes/


blurred_test  dev.jsonl  img  LICENSE.txt  README.md  test.jsonl  train.jsonl
